# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [22]:
import string

def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    # TODO
    def normalize(text):
        norm = str()
        for char in text:
            if char not in string.punctuation:
                norm += char
        norm = norm.lower()
        return norm.strip().split(' ')
    reference_words = normalize(reference_text)
    recognized_words = normalize(recognized_text)
    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            #TODO
            m = 0 if (reference_words[i-1] == recognized_words[j-1]) else 1                
            distance_matrix[i][j] = min(distance_matrix[i-1][j] + 1, distance_matrix[i][j-1] + 1, distance_matrix[i-1][j-1]+m)

    # Расчет WER  (в процентах)
    wer = 100 * distance_matrix[-1][-1]/len(reference_words)
    return wer

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [23]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [5]:
!pip install tabulate
from tabulate import tabulate
# используйте tabulate для отладки

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [162]:

def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    
    # Перенесите сюда код из задания 1.a.
    #wer = 
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    # TODO
    def normalize(text):
        norm = str()
        for char in text:
            if char not in string.punctuation:
                norm += char
        norm = norm.lower()
        return norm.strip().split(' ')
    reference_words = normalize(reference_text)
    recognized_words = normalize(recognized_text)
    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            #TODO
            m = 0 if (reference_words[i-1] == recognized_words[j-1]) else 1                
            distance_matrix[i][j] = min(distance_matrix[i-1][j] + 1, distance_matrix[i][j-1] + 1, distance_matrix[i-1][j-1]+m)

    # Расчет WER  (в процентах)
    wer = 100 * distance_matrix[-1][-1]/len(reference_words)
    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 
    #TODO
    # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация 
    correct = 0
    deletion = 0
    insertion = 0
    substitution = 0
    ali = [[],[],[]]
    
    i = len(reference_words)
    j = len(recognized_words)
    step = 0
    while i > 0 or j > 0:
        cur_i = reference_words[max(i-1, 0)]
        cur_j = recognized_words[max(j-1, 0)]
        if cur_i == cur_j:
            #correct
            ali[0] = [cur_i] + ali[0]
            ali[1] = [cur_j] + ali[1]
            ali[2] = ['C'] + ali[2]
            i = max(0, i-1)
            j = max(0, j-1)
            correct += 1
        else:
            if distance_matrix[i][j] == distance_matrix[i-1][j-1] + 1:
                #substitution
                ali[0] = [cur_i] + ali[0]
                ali[1] = [cur_j] + ali[1]
                ali[2] = ['S'] + ali[2]
                i = max(0, i-1)
                j = max(0, j-1)
                substitution += 1
            elif distance_matrix[i][j] == distance_matrix[i][j-1] + 1:
                #insertion
                ali[0] = ['***'] + ali[0]
                ali[1] = [cur_j] + ali[1]
                ali[2] = ['I'] + ali[2]
                j = max(0, j-1)
                insertion += 1
            elif distance_matrix[i][j] == distance_matrix[i-1][j] + 1:
                #deletion
                ali[0] = [cur_i] + ali[0]
                ali[1] = ['***'] + ali[1]
                ali[2] = ['D'] + ali[2]
                deletion += 1
                i = max(0, i-1)
            else: 
                print('error')
        step +=1
            
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    print(tabulate(ali))
    
    return {"wer" : wer,
            "cor": correct, 
            "del": deletion,
            "ins": insertion,
            "sub": substitution,
            "ali": ali}


In [163]:
calculate_wer_with_alignment('привет', 'привет студент')
#def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
#     return {
#             "wer" : 0,
#             "cor": 2, 
#             "del": 0,
#             "ins": 0,
#             "sub": 0,
#             "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]}

------  -------
привет  ***
привет  студент
C       I
------  -------


{'wer': 100.0,
 'cor': 1,
 'del': 0,
 'ins': 1,
 'sub': 0,
 'ali': [['привет', '***'], ['привет', 'студент'], ['C', 'I']]}

In [164]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

------  -------
привет  студент
привет  студент
C       C
------  -------
------  -------
привет  студент
***     студент
D       C
------  -------
------  -------
привет  ***
привет  студент
C       I
------  -------
------  -------
привет  студент
пока    студент
S       C
------  -------
Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно RichTranscriptErrorRate (RTER) по формуле

$$ RTER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [175]:
def calculate_wer_per(reference_text: str, recognized_text: str):
    punct = string.punctuation 
    # Приведение текста к нижнему регистру, разбивка на слова c сохранением пунктуации
    # TODO
    def normalize_save_punkt(text):
        norm = str()
        for char in text:
            if char not in punct:
                norm += char
            else: 
                norm += ' ' + char + ' '
        norm = norm.lower()
        norm = norm.strip().split(' ')
        return [el for el in norm if el != '']
    reference_words = normalize_save_punkt(reference_text)
    recognized_words = normalize_save_punkt(recognized_text)
#     print(reference_words)
#     print(recognized_words)
    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            #TODO
            m = 0 if (reference_words[i-1] == recognized_words[j-1]) else 1  
            if reference_words[i-1] in string.punctuation and recognized_words[j-1] not in string.punctuation:
                m = 800
            if reference_words[i-1] not in string.punctuation and recognized_words[j-1] in string.punctuation:
                m = 800
            distance_matrix[i][j] = min(distance_matrix[i-1][j] + 1, distance_matrix[i][j-1] + 1, distance_matrix[i-1][j-1]+m)

    
    #print(distance_matrix)
    correct = 0
    c_p = 0
    deletion = 0
    d_p = 0
    insertion = 0
    i_p = 0
    substitution = 0
    s_p = 0
    ali = [[],[],[]]
    
    # провереяет, являются ли 2 морфемы знаками пунктуации/словами/знаком и словом
    def check(mor1, mor2):
        if mor1 in punct and mor2 in punct:
            return 0
        elif mor1 not in punct and mor2 not in punct:
            return 1
        else: 
            return 2
    
    i = len(reference_words)
    j = len(recognized_words)
    step = 0
    while i > 0 or j > 0: 
        cur_i = reference_words[max(i-1, 0)]
        cur_j = recognized_words[max(j-1, 0)]
        if cur_i == cur_j:
            #correct
            ali[0] = [cur_i] + ali[0]
            ali[1] = [cur_j] + ali[1]
            if cur_i not in punct:
                ali[2] = ['C'] + ali[2]
                correct += 1
                #print(f'step {step} i {i} j {j} C')
            else:
                ali[2] = ['C_p'] + ali[2]
                #print(f'step {step} i {i} j {j} Cp')
                c_p += 1
            i = max(0, i-1)
            j = max(0, j-1)  
        else:
            if distance_matrix[i][j] == distance_matrix[i-1][j-1] + 1 and check(cur_i, cur_j) == 0:
                #substitution
                    ali[2] = ['S_p'] + ali[2]
                    s_p +=1
                    ali[0] = [cur_i] + ali[0]
                    ali[1] = [cur_j] + ali[1]
                    i = max(0, i-1)
                    j = max(0, j-1)
                    #print(f'step {step} i {i} j {j} Sp')
            elif distance_matrix[i][j] == distance_matrix[i-1][j-1] + 1 and check(cur_i, cur_j) == 1:
                    ali[2] = ['S'] + ali[2]
                    substitution += 1
                    ali[0] = [cur_i] + ali[0]
                    ali[1] = [cur_j] + ali[1]
                    i = max(0, i-1)
                    j = max(0, j-1)
                    #print(f'step {step} i {i} j {j} S')     
            elif distance_matrix[i][j] == distance_matrix[i-1][j] + 1:
                #deletion
                ali[0] = [cur_i] + ali[0]
                ali[1] = ['***'] + ali[1]
                if cur_i not in punct:
                    ali[2] = ['D'] + ali[2]
                    deletion += 1
                    #print(f'step {step} i {i} j {j} D')
                else: 
                    ali[2] = ['D_p'] + ali[2]
                    d_p += 1
                    #print(f'step {step} i {i} j {j} Dp')
                i = max(0, i-1) 
            elif distance_matrix[i][j] == distance_matrix[i][j-1] + 1:
                #insertion
                ali[0] = ['***'] + ali[0]
                ali[1] = [cur_j] + ali[1]
                if cur_j not in punct:
                    ali[2] = ['I'] + ali[2]
                    insertion += 1
                    #print(f'step {step} i {i} j {j} I')
                else: 
                    ali[2] = ['I_p'] + ali[2]
                    i_p += 1
                    #print(f'step {step} i {i} j {j} Ip')
                j = max(0, j-1)    
                
            else: 
                print('error')
        step +=1
        
    #print(distance_matrix)        
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    wer = 100 * (insertion + deletion + substitution)/(deletion + substitution + correct)
    per = 100* (i_p + d_p + s_p)/(d_p + s_p + c_p)
    
    print(tabulate(ali))
    return {"wer" : wer,
            "per": per, 
            "ali": ali}


In [176]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

------  -------  ---
привет  студент  .
привет  студент  ***
C       C        D_p
------  -------  ---
------  -------  ---
привет  студент  .
***     студент  .
D       C        C_p
------  -------  ---
------  ---  -------  ---
привет  ***  студент  .
привет  .    студент  ***
C       I_p  C        D_p
------  ---  -------  ---
---  ------  -------  ---
***  привет  студент  .
.    ***     студент  ?
I_p  D       C        S_p
---  ------  -------  ---
Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [210]:
import itertools

def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)
    
    reference_words = reference_text
    recognized_words = recognized_text
    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER
    sawer=0
    ali=[]
    
    # distance_matrix
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            m = 0 if (reference_words[i-1] == recognized_words[j-1]) else 1                
            distance_matrix[i][j] = min(distance_matrix[i-1][j] + 1, distance_matrix[i][j-1] + 1, distance_matrix[i-1][j-1]+m)
    
    correct = 0
    deletion = 0
    insertion = 0
    substitution = 0
    ali = [[],[],[]]
    
    i = len(reference_words)
    j = len(recognized_words)
    step = 0
    while i > 0 or j > 0:
        cur_i = reference_words[max(i-1, 0)]
        cur_j = recognized_words[max(j-1, 0)]
        if cur_i == cur_j:
            #correct
            ali[0] = [cur_i] + ali[0]
            ali[1] = [cur_j] + ali[1]
            ali[2] = ['C'] + ali[2]
            i = max(0, i-1)
            j = max(0, j-1)
            correct += 1
        else:
            if distance_matrix[i][j] == distance_matrix[i-1][j-1] + 1:
                #substitution
                ali[0] = [cur_i] + ali[0]
                ali[1] = [cur_j] + ali[1]
                ali[2] = ['S'] + ali[2]
                i = max(0, i-1)
                j = max(0, j-1)
                substitution += 1
            elif distance_matrix[i][j] == distance_matrix[i][j-1] + 1:
                #insertion
                ali[0] = ['***'] + ali[0]
                ali[1] = [cur_j] + ali[1]
                ali[2] = ['I'] + ali[2]
                j = max(0, j-1)
                insertion += 1
            elif distance_matrix[i][j] == distance_matrix[i-1][j] + 1:
                #deletion
                ali[0] = [cur_i] + ali[0]
                ali[1] = ['***'] + ali[1]
                ali[2] = ['D'] + ali[2]
                deletion += 1
                i = max(0, i-1)
            else: 
                print('error')
        step +=1
            
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    print(tabulate(ali))
    
    sawer = 1000000
    permuts = list(itertools.permutations(set(recognized_speakers)))
    for permut in permuts:
        print(permut, "permut")
        ref = {}
        for i, spk in enumerate(set(reference_speakers)): 
            try:
                ref[spk] = permut[i]
            except:     
                ref[spk] = None
        mapping = [ref[spk] for spk in reference_speakers]
        print(mapping, "mapping")
        print(recognized_speakers, 'recognized_speakers')
        print(ali[2], "ali")
        # подсчет I_s
        I_s = 0
        for i in range(len(mapping)):
            try:
                if mapping[i] != recognized_speakers[i] and ali[2][i] == 'C':
                    I_s += 1
            except:
                ...
        print(I_s, "I_s")
        ans = 100 * (insertion + deletion + substitution + I_s)/(deletion + substitution + correct + I_s)
        
        if ans < sawer:
            sawer = ans
            
    print(f"sawer: {sawer}")        
    return {"sawer" : sawer, 
            "ali": ali}


In [212]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
     "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

------  -------
привет  студент
привет  студент
C       C
------  -------
(1, 2) permut
[2, 1] mapping
[1, 2] recognized_speakers
['C', 'C'] ali
2 I_s
(2, 1) permut
[1, 2] mapping
[1, 2] recognized_speakers
['C', 'C'] ali
0 I_s
sawer: 0.0
------  -------
привет  студент
привет  студент
C       C
------  -------
(1, 2) permut
[1, 1] mapping
[1, 2] recognized_speakers
['C', 'C'] ali
1 I_s
(2, 1) permut
[2, 2] mapping
[1, 2] recognized_speakers
['C', 'C'] ali
1 I_s
sawer: 33.333333333333336


AssertionError: 